# Discrete-time LQ-optimal control on an infinite interval

The goal is to find a sequence $\mathbf{u}_{0},\ldots,\mathbf{u}_{N-1},\mathbf{u}_{N},\ldots$ that yields the minimum in the following problem

\begin{align*}
\min_{\mathbf{x}_{1},\ldots,\mathbf{x}_{N},\mathbf{u}_{0},\ldots,\mathbf{u}_{N-1}} &\; \frac{1}{2}\sum_{k=0}^{\infty}\left[\mathbf x_k^T \mathbf Q \mathbf x_k+\mathbf u_k^T \mathbf R\mathbf u_k\right]\\
\text{s.t. } &\; \mathbf x_{k+1} = \mathbf A \mathbf x_{k} + \mathbf B \mathbf u_k,\\
& \mathbf x_0 = \mathbf r_0,\\
& \mathbf Q\geq 0, \mathbf R>0.
\end{align*}

Obviously, finding an infinite sequence is not quite feasible as a search over the (infinite number of the) infinite values. Instead, some trick needs to be invoked. Here the trick is to extend the discrete-time Riccati equation over an infinite interval and turn it into a (discrete-time) algebraic Riccati equation (DARE).

\begin{equation}
\mathbf S=\mathbf A^T\left[\mathbf S-\mathbf S\mathbf B(\mathbf B^T\mathbf S\mathbf B+\mathbf R)^{-1}\mathbf B^T\mathbf S\right]\mathbf A+\mathbf Q
\end{equation}

This equation can be solved in two ways. First, as the limiting solution to the discrete-time (recurrent) Riccati equation. Second, directly as a special (in fact quadratic) equation with matrices. Below we demonstrate both.

Once we have the solution to DARE, the optimal control is given as

\begin{equation}
 \mathbf K = (\mathbf R + \mathbf B^T\mathbf S\mathbf B)^{-1}\mathbf B^T\mathbf S\mathbf A.
\end{equation}

## Solving DARE as a limit of discrete-time Riccati equation 

We reuse the code developed previously

In [8]:
function dre(A,B,Q,R,SN,N)
    nx = size(A,1)
    nu = size(B,2)
    S = [Matrix{Float64}(undef, nx, nx) for _ in 0:N]
    K = [Matrix{Float64}(undef, nu, nx) for _ in 0:N-1]
    S[end] = SN # note that unlike in the mathematical formula, here the final time is at index N+1
    for k = N:-1:1
        K[k] = (R+B'*S[k+1]*B)\B'*S[k+1]*A
        S[k] = A'*S[k+1]*(A-B*K[k])+Q
    end
    return S,K
end

dre (generic function with 1 method)

Now some data (system and the cost)

In [10]:
A = [1.0 2; 3 4]
B = [5.0 6; 7 8]
Q = [1.0 0; 0 100]
R = [1.0 0; 0 1]

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [11]:
SN = [1.0 0; 0 100]

2×2 Array{Float64,2}:
 1.0    0.0
 0.0  100.0

Let's set the length of the time interal high enough. In principle, a full algorithm should do some iterations untill no change is observed.

In [17]:
N = 100

100

In [18]:
S,K = dre(A,B,Q,R,SN,N);

In [20]:
S∞ = S[1]

2×2 Array{Float64,2}:
 10.8225     7.69816
  7.69816  106.058  

Keep an eye on this value untill we provide another solution method.

## Solving DARE directly as an algebraic equation

### Scalar DARE

The fact that DARE is actually a quadratic equation need not be immediately obvious. Let's have a look at the scalar case first

\begin{equation}
 s = q + a^2s - \frac{a^2b^2s}{b^2s+r},
\end{equation}

which by multiplying both sides by the denominator turns into

\begin{equation}
 s(b^2s+r) = q(b^2s+r) + a^2s(b^2s+r) - a^2b^2s.
\end{equation}

Grouping together the coefficients with equal powers of $s$ yields the quadratic equation in the standard form

\begin{equation}
b^2s^2 + (r - a^2r - b^2q)s - rq =0.
\end{equation}

In [2]:
a = 1/2;
b = 1;
q = 1;
r = 1;

In [3]:
s2 = b^2
s1 = (r - a^2*r - b^2*q)
s0 = -r*q

-1

In [4]:
using Polynomials

In [5]:
a = Poly([s0, s1, s2],:s)

Poly(-1.0 - 0.25*s + 1.0*s^2)

In [6]:
s = roots(a)

2-element Array{Float64,1}:
 -0.8827822185373188
  1.1327822185373186

The positive solution is the second one, that is

In [7]:
s[2]

1.1327822185373186

While we are at it, let's explore the solvability conditions a bit.

Trivial analysis shows that for $q=0$, one of the roots is $s_1=0$ and the other is always $s_2<0$. Hence the solution of DARE that represents the steady-state solution of the recurrent Riccati equation is $s=0$. As a consequence, the optimal state-feedback gain is $k=0$. For an unstable system this would be unacceptable but for a stable system this seems acceptable: the system is stable even withouth the control, therefore, when the state is not penalized in the criterion at all ($q=0$), the optimal strategy is not regulating at all. Mathematically correct. Nonethelesss, from an engineering viewpoint we may be quite unhappy because the role of the feedback regulator is also to attenuate the influence of external disturbances. Our optimal state-feedback regulator does not help at all in these situation. That is why we may typically want to require a positive definite solution of DARE. And that is generally guaranteed if $(\mathbf A,\mathbf Q)$ is observable and not just detectable.

### Matrix DARE

We use the implementation of a solver for DARE available in the [ControlSystems](https://github.com/JuliaControl/ControlSystems.jl) package. A more recent alternative is the [MatrixEquations](https://github.com/andreasvarga/MatrixEquations.jl) package. 

In [25]:
using ControlSystems

In [26]:
S = dare(A,B,Q,R)

2×2 Array{Float64,2}:
 10.8225     7.69816
  7.69816  106.058  

Now check if this result agrees with the limiting solution to the recurrent Riccati equation.

In [24]:
S-S∞

2×2 Array{Float64,2}:
 3.56692e-12  -6.0485e-12 
 2.77822e-12  -4.78906e-12

## Optimal state feedback

Whichever way we solved the DARE equations, once we have it, the LTI state feedback is

In [27]:
K = (B'*S*B + R)\(B'*S*A)

2×2 Array{Float64,2}:
  1.32519    1.1356  
 -0.799127  -0.505034

Done. But a more complete design (including simulations and discussions of a choice of the weight matrices $\mathbf Q$ and $\mathbf R$) is demonstrated in the next notebook.

## References

[1] Lewis, Frank, Draguna Vrabie, and Vassilis L. Syrmos. Optimal Control. 3rd edition. Hoboken: Wiley, 2012.

[2] Rawlings, James B., David Q. Mayne, and Moritz M. Diehl. Model Predictive Control: Theory, Computation, and Design. 2nd ed. Madison, Wisconsin: Nob Hill Publishing, LLC, 2017.